# Day 16
## Puzzle 1

In [124]:
import numpy as np
import networkx as nx

In [125]:
input_file = 'input_1.txt'
# input_file = 'test_input_1.txt'

Read input reindeer maze.

In [126]:
with open(file=input_file, mode="r") as file:
    reindeer_maze = []

    for line in file:
        reindeer_maze.append(list(line.strip()))

    reindeer_maze_matrix = np.matrix(reindeer_maze)

In [127]:
reindeer_maze_matrix

matrix([['#', '#', '#', ..., '#', '#', '#'],
        ['#', '.', '.', ..., '.', 'E', '#'],
        ['#', '.', '#', ..., '#', '.', '#'],
        ...,
        ['#', '#', '#', ..., '#', '.', '#'],
        ['#', 'S', '.', ..., '#', '.', '#'],
        ['#', '#', '#', ..., '#', '#', '#']], dtype='<U1')

Each tile gets 4 nodes - one for each direction (North, East, South, West). The cost for turning from East to North on tile (i, j) is represented by a weighted directed edge between node (i, j, 'East') to (i, j, 'North') with a weight of 1000. Adjacent tiles only connect in the "same direction", i.e. (i, j, 'North') connects to (i - 1, j, 'North') with a weight of 1 (the tiles also have the connection (i - 1, j, 'South') connects to (i, j, 'South') if the raindeers are heading in the opposite direction).

In [128]:
m, n = reindeer_maze_matrix.shape
start_node = None
end_nodes = None
weighted_edges = []

for i in range(m):
    for j in range(n):
        current_node_value = reindeer_maze_matrix[i, j]

        if current_node_value == 'S':
            start_node = (i, j, 'East')

        elif current_node_value == 'E':
            end_nodes = [(i, j, 'North'), (i, j, 'East'), (i, j, 'South'), (i, j, 'West')]

        if current_node_value != '#':
            current_node_weighted_edges = [
                ((i, j, 'North'), (i, j, 'East'), 1000),
                ((i, j, 'East'), (i, j, 'North'), 1000),
                ((i, j, 'East'), (i, j, 'South'), 1000),
                ((i, j, 'South'), (i, j, 'East'), 1000),
                ((i, j, 'South'), (i, j, 'West'), 1000),
                ((i, j, 'West'), (i, j, 'South'), 1000),
                ((i, j, 'West'), (i, j, 'North'), 1000),
                ((i, j, 'North'), (i, j, 'West'), 1000)
            ]
            weighted_edges.extend(current_node_weighted_edges)

            if j + 1 < m:
                current_east_node_value = reindeer_maze_matrix[i, j + 1]

                if current_east_node_value != '#':
                    weighted_edges.append(((i, j, 'East'), (i, j + 1, 'East'), 1))
                
            if j - 1 >= 0:
                current_west_node_value = reindeer_maze_matrix[i, j - 1]

                if current_west_node_value != '#':
                    weighted_edges.append(((i, j, 'West'), (i, j - 1, 'West'), 1))

            if i + 1 < n:
                current_south_node_value = reindeer_maze_matrix[i + 1, j]

                if current_south_node_value != '#':
                    weighted_edges.append(((i, j, 'South'), (i + 1, j, 'South'), 1))

            if i - 1 >= 0:
                current_north_node_value = reindeer_maze_matrix[i - 1, j]

                if current_north_node_value != '#':
                    weighted_edges.append(((i, j, 'North'), (i - 1, j, 'North'), 1))

Create a weighted directed graph.

In [129]:
D = nx.DiGraph()
D.add_weighted_edges_from(weighted_edges)

Find the shortest path length. Since we don't know which direction the reindeers will face on the end tile "E", we have 4 shortest paths to calculate (1 for each end tile direction) and then choose the shortest one.

In [130]:
path_lengths = []

for end_node in end_nodes:
    path_lengths.append(nx.shortest_path_length(D, source=start_node, target=end_node, weight='weight'))

In [131]:
min(path_lengths)

98416

## Puzzle 2

Now, we generate all the shortest paths between the start tile "S" and the end tile "E" and count the number of tiles that are traversed.

In [132]:
tiles_in_optimal_paths = set()

for end_node in end_nodes:
    for path in nx.all_shortest_paths(D, source=start_node, target=end_node, weight='weight'):
        for node in path:
            tiles_in_optimal_paths.add((node[0], node[1]))

In [133]:
len(tiles_in_optimal_paths)

471